In [2]:
import random
import numpy as np


def short(line):
    source = line.split("\t")[0].replace("(", " ").replace(")", " ").replace(",", "").split()
    return len([w for w in source if w.lower() == w]) == 2


random.seed(1)
np.random.seed(1)

train = list(zip([l.strip() for l in open("systematicity/train.src").readlines()],
            [l.strip() for l in open("systematicity/train.tgt").readlines()]))
train = [p for p in train if "reverse echo" not in p[0] and \
         "prepend remove_first" not in p[0] and "echo remove_first" not in p[0] and \
         "prepend reverse" not in p[0]]
n = int(len(train) * 0.9)
test = train[n:]
train = train[:n]

sys_overgen = open("regenerate/sys_overgen.txt.txt").readlines()

sys_overgen_short = [l for l in sys_overgen if short(l)]
sys_overgen_long = [l for l in sys_overgen if not short(l)]

random.shuffle(sys_overgen_short)
random.shuffle(sys_overgen_long)

mini = int(len(train) * 0.0025)
tiny = int(len(train) * 0.005)
small = int(len(train) * 0.01)
medium = int(len(train) * 0.02)
regular = int(len(train) * 0.04)
regular2 = int(len(train) * 0.08)

In [3]:
len(sys_overgen_short)

base_examples = random.sample(sys_overgen_short, int(mini/2)) + \
    random.sample(sys_overgen_long, int(mini/2))
with open("systematicity_overgeneralisation/test_overgeneralisation.src", 'w') as f_src, \
    open("systematicity_overgeneralisation/test_overgeneralisation.original.tgt", 'w') as f_tgt_org, \
    open("systematicity_overgeneralisation/test_overgeneralisation.exception.tgt", 'w') as f_tgt_exc:
    for l in base_examples:
        source, tgt_org, tgt_exc = l.split("\t")
        f_src.write(source.replace(" (", "").replace(" )", "") + '\n')
        f_tgt_org.write(tgt_org + '\n')
        f_tgt_exc.write(tgt_exc.strip() + '\n')

In [4]:
import random
import numpy as np

for seed in range(11, 21):
    random.seed(seed)
    np.random.seed(seed)
    for size, name in [(mini, 0.0025), (0, 0), (tiny, 0.005), (small, 0.01), (medium, 0.02), (regular, 0.04),\
                       (regular2, 0.08)]:
        if name == 0:
            overgen_samples = []
        elif name == 0.0025:
            overgen_samples = base_examples
        else:
            overgen_samples = \
                base_examples + \
                random.sample([s for s in sys_overgen_short if s not in base_examples], int(size/2)) + \
                random.sample([s for s in sys_overgen_long if s not in base_examples], int(size/2))
        overgen_samples = [
            (l.split("\t")[0].replace(" (", "").replace(" )", ""), l.split("\t")[2].strip())
            for l in overgen_samples]

        dataset = train + overgen_samples
        random.shuffle(dataset)

        with open(f"systematicity_overgeneralisation/train_{name}_{seed}.src", 'w') as f_src, \
             open(f"systematicity_overgeneralisation/train_{name}_{seed}.tgt", 'w') as f_tgt:
            for src, tgt in dataset:
                f_src.write(src.strip() + '\n')
                f_tgt.write(tgt.strip() + '\n')

In [4]:
with open(f"systematicity_overgeneralisation/test_iid.src", 'w') as f_src, \
    open(f"systematicity_overgeneralisation/test_iid.tgt", 'w') as f_tgt:
    for src, tgt in test:
        f_src.write(src.strip() + '\n')
        f_tgt.write(tgt.strip() + '\n')